contract store_value(value: int){

    let storage: int = 0;

    entry replace(new_value: int){
       
        storage = new_value;
    } 
}

In [1]:
from lexer.tzscript_lexer import TzScriptLexer
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry
from parser.slr_parser import SLR1Parser, build_slr_ast
words_separated_by_spaces = ['contract', 'store_value','(','value',':', 'int',')','{','let', 'storage',':', 'int','=', '0',';', 'entry', 'replace','(','new_value',':', 'int',')','{','storage', '=', 'new_value',';','}','}']
table = [(TZSCRIPT_GRAMMAR[tok], tok) for tok in words_separated_by_spaces]

expected_tokens = [ Token(x[1], x[0]) for x in table]

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)
tokens = [Token('contract',contract),Token('store_value',idx),Token('(',opar),Token('value',idx),Token(':',colon),Token('int',typex),Token(')',cpar),Token('{',ocur),Token('let',let),Token('storage',idx),Token(':',colon),Token('int',typex),Token('=',equal),Token('0',num),Token(';',semi),Token('entry',entry),Token('replace',idx),Token('(',opar),Token('new_value',idx),Token(':',colon),Token('int',typex),Token(')',cpar),Token('{',ocur),Token('storage',idx),Token('=',equal),Token('new_value',idx),Token(';',semi),Token('}',ccur),Token('}',ccur),Token('EOF',TZSCRIPT_GRAMMAR.EOF)]

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)


self,  <program>
other (contract id ( <param-list> ) { <stat_list> }, <function <lambda> at 0x7f9340a73820>, None, None, None, None, None, None, None, None)
self,  <stat_list>
other (<stat> <stat_list>, <function <lambda> at 0x7f9340a7a9d0>, None, None)
self,  <stat_list>
other ('<stat>', <function <lambda> at 0x7f9340a7f040>, None)
self,  <stat>
other ('<def-func>', <function <lambda> at 0x7f9340a7f550>, None)
self,  <stat>
other ('<def-entry>', <function <lambda> at 0x7f9340a7fa60>, None)
self,  <stat>
other ('<if-stat>', <function <lambda> at 0x7f9340a7ff70>, None)
self,  <stat>
other ('<else-stat>', <function <lambda> at 0x7f9340a8d4c0>, None)
self,  <stat>
other ('<var-call>', <function <lambda> at 0x7f9340a8d9d0>, None)
self,  <stat>
other ('<let-var>>', <function <lambda> at 0x7f9340a8dee0>, None)
self,  <if-stat>
other (if ( <expr> ) { <stat_list> }, <function <lambda> at 0x7f9340ad1430>, None, None, None, None, None, None, None)
self,  <else-stat>
other (else { <stat_list> }, 

In [2]:
productions, operations = derivation

operations


['SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE']

In [3]:
ast = build_slr_ast(productions, operations, tokens)

$: EOF


In [4]:

from visitors.type_check_visitor import TypeCheckVisitor
type_visitor = TypeCheckVisitor()

In [5]:
type_visitor.visit_program(ast)

In [6]:
from visitors.scope_check_visitor import ScopeCheckVisitor
scope_visitor = ScopeCheckVisitor()

In [7]:
scope_visitor.visit_program(ast)

In [8]:
from visitors.string_rep_visitor import StringReprVisitor
string_visitor = StringReprVisitor()

In [9]:
string_visitor.visit_program(ast)

In [10]:
print(string_visitor.result)

program store_value(value):
    int storage =  0

replace( int new_value):
	storage =  new_value


